In [ ]:
import pandas as pd
import numpy as np
import os,sys
import json
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/train.csv')
test_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/test.csv') 



In [ ]:
#data_train_t1_en = train_df['title1_en']
#data_train_t2_en = train_df['title2_en']

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
data_text_t1_en = train_df[['title1_en']]  #+ test_df[['title1_en']]
data_text_t1_en['index'] = train_df.index
documents_t1 = data_text_t1_en

In [ ]:
data_text_t2_en = train_df[['title2_en']]
data_text_t2_en['index'] = train_df.index
documents_t2 = data_text_t2_en

In [ ]:
type(documents_t1)

In [ ]:
#doc_sample_t1 = documents_t1[documents_t1['index'] == 0].values[0][0]
#print('original document: ')
#words = []
stemmer = SnowballStemmer("english")
#stemmer = PorterStemmer()
#for word in doc_sample_t1.split(' '):
 #   words.append(word)
#print(words)
#print('\n\n tokenized and lemmatized document: ')
#print(preprocess(doc_sample_t1))

In [ ]:
processed_docs_t1 = documents_t1['title1_en'].map(preprocess)
processed_docs_t1[:10]

In [ ]:
len(processed_docs_t2)

In [ ]:
processed_docs_t2 = documents_t2['title2_en'].map(preprocess)
processed_docs_t2[:10]

In [ ]:
# Combine both the dictionaries
processed_docs = processed_docs_t1 + processed_docs_t2


In [ ]:
processed_docs[:10]

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)

In [ ]:
len(dictionary)

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

In [ ]:
bow_corpus[0]

In [ ]:
len(bow_corpus)

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
len(bow_corpus)

In [ ]:

for k, v in dictionary.iteritems():
    print(k, v)

In [ ]:
len(dictionary)

In [ ]:
docs = documents_t1['title1_en'] + documents_t2['title2_en']

In [ ]:
type(docs[0])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline


vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)

svd_model = TruncatedSVD(n_components=100,         #num dimensions
                         algorithm='randomized',
                         n_iter=10)

svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])

svd_matrix = svd_transformer.fit_transform(docs)

In [ ]:
type(svd_matrix)

In [ ]:
X = svd_matrix

In [ ]:
 y = train_df.loc[:,'label']

In [ ]:
y

In [ ]:
y_new =[0] * len(y)
for i in range(len(y)):
    if y[i] == 'unrelated':
        y_new[i] = 0
    elif y[i] == 'agreed':
        y_new[i] = 1
    elif y[i] == 'disagreed':
        y_new[i] = 2

In [ ]:
y_new

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_new, test_size = 0.25, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

In [ ]:
confusion_matrix?

In [ ]:
51815 + 4333 + 52

In [ ]:
sum(cm)

In [ ]:
sum(sum(cm))

In [ ]:
56200/80138

In [ ]:
type(X_test)

In [ ]:
test_df

In [ ]:
y_pred = classifier.predict()

In [ ]:
data_text_t1_en_test = test_df[['title1_en']]
data_text_t1_en_test['index'] = test_df.index
documents_t1_test = data_text_t1_en_test
data_text_t2_en_test = test_df[['title2_en']]
data_text_t2_en_test['index'] = test_df.index
documents_t2_test = data_text_t2_en_test

In [ ]:
len(documents_t2_test)

In [ ]:
docs_test = documents_t1_test['title1_en'] + documents_t2_test['title2_en']

In [ ]:


svd_matrix_test = svd_transformer.fit_transform(docs_test)

In [ ]:
svd_matrix_test

In [ ]:
X_test

In [ ]:
y_pred = classifier.predict(svd_matrix_test)

In [ ]:
len(y_pred)

In [ ]:
y_pred

In [ ]:
ss_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/sample_submission.csv') 

In [ ]:
ss_df

In [ ]:
y_pred_final = [0] * len(y_pred)
for i in range(len(y_pred)):
    if y_pred[i] == 0:
        y_pred_final[i] ='unrelated'
    elif y_pred[i] == 1:
        y_pred_final[i] = 'agreed'
    elif y_pred[i] == 2:
        y_pred_final[i] = 'disagreed'   

In [ ]:
for i in range(len(y_pred_final)):
    if y_pred_final[i] == 'unrelated':
        print(y_pred_final[i])

In [ ]:
y_pred_sub = [0] * len(y_pred)
for i in range(len(y_pred)):
    y_pred_sub[i] = [0] * 2

In [ ]:
test_df

In [ ]:
for i in range(len(y_pred_sub)):
    y_pred_sub[i][0] = test_df.iloc[i,0]

In [ ]:
for i in range(len(y_pred_sub)):
    y_pred_sub[i][1] = y_pred_final[i]

In [ ]:
y_pred_sub 

In [ ]:
out = pd.DataFrame(y_pred_sub, columns = ["Id", "Category"])

In [ ]:
out

In [ ]:
out.to_csv('/Users/Administrator/Downloads/FakeNewsWSDM/output.csv', index = False)